### Librerias

In [2]:
import zulip # Conexión a API de Zulip
import re # Expresiones regulares
import nltk # Procesamiento de lenguaje (tokenización)
import random # Generar números aleatorios
from bs4 import BeautifulSoup # Procesar respuesta HTML como texto plano

### API Request a Zulip

In [6]:
# Reemplazar con sus datos y clave de API

client = zulip.Client(
    api_key="123456"
    ,email="tuemail@tuemail.com"
    ,site="https://labo2023v.zulip.rebelare.com"
)

request = {
    "anchor": "newest",
    "num_before": 100,
    "num_after": 0,
    "narrow": [
        {"operator": "stream", "operand": "z-CheckIn"},
    ],
}
result = client.get_messages(request)['messages']

### Patterns Regex

In [7]:
pattern = re.compile(r'[0-9]{6}[^\d]{2,5}[0-9]{6}[^\d]{2,5}[0-9]{6}[^\d]{2,5}[0-9]{6}[^\d]{2,5}[0-9]{6}',re.S)

pattern2 = re.compile(r'(?<!\d)\d{6}[^\d]{0,7}\d{6}[^\d]{0,7}\d{6}[^\d]{0,7}\d{6}[^\d]{0,7}\d{6}(?!\d)',re.S)

### Busqueda de secuencia

In [8]:
v_dict = {}
  
for reg in result:
  texto_limpio = BeautifulSoup(reg['content'], "html.parser").get_text()
  numbers = re.findall(pattern2, texto_limpio.replace("\n", " ").replace('100003','').replace('999983','').replace(',',''))
  creator = reg['sender_full_name']
  v_dict[creator] = numbers

### Resultados preliminares

In [9]:
# Si aparecen listas vacias se trata de condiciones no satisfechas por la expresión regular o algún mensaje producto de una respuesta y no un mensaje original
v_dict

{'Notification Bot': [],
 'Marcos Buccellato': ['771349 771359 771389 771401 771403'],
 'Fernando Coz': ['987705 163855 325599 590517 269257'],
 'mariano daniel marchetta': ['100019 - 100043 - 100049 - 100057 - 100069'],
 'Fernando Bogorni': ['100103 100109 100129 100151 100153'],
 'Lucas Cardenas': ['107897 108089 108377 108413 108881'],
 'Marcela Distefano': ['100019 100043 100049 100057 100069'],
 'Hernan Ifran': ['100591 100609 100613 100621 100649'],
 'Verónica García Río': ['100169 100183 100189 100193 100207'],
 'Damian Joglar': ['200003 - 200009 - 200013 - 220003  - 200023'],
 'Aureliano Chavarría': ['127031 262151 398039 524287 786433'],
 'Luis Darío De Angeli': [],
 'Leandro.Cerutti': [],
 'Gustavo Denicolay': [],
 'Esteban Elia': ['677213 727817 311237 660719 106427'],
 'Karina L. Roitman': ['201107  433781 590041 604711 931517'],
 'Johanna Noval': ['100333- 100999 – 120011 - 800029 – 900091'],
 'Milagros Rodríguez Saá': ['100019 200017 300043 400093 500083'],
 'Carla Fiori'

### Tokenizo las listas por tener distintos formatos

In [10]:
for key in v_dict:
    tokens = []
    for elemento in v_dict[key]:
        tokens_raw = nltk.word_tokenize(elemento)
        tokens_limpios = [re.sub(r'[^\w\s]','',token) for token in tokens_raw if not token.isspace() and token != '' and len(token) >= 6]
        tokens.extend(tokens_limpios)
        v_dict[key] = list(map(int, tokens))

v_dict

{'Notification Bot': [],
 'Marcos Buccellato': [771349, 771359, 771389, 771401, 771403],
 'Fernando Coz': [987705, 163855, 325599, 590517, 269257],
 'mariano daniel marchetta': [100019, 100043, 100049, 100057, 100069],
 'Fernando Bogorni': [100103, 100109, 100129, 100151, 100153],
 'Lucas Cardenas': [107897, 108089, 108377, 108413, 108881],
 'Marcela Distefano': [100019, 100043, 100049, 100057, 100069],
 'Hernan Ifran': [100591, 100609, 100613, 100621, 100649],
 'Verónica García Río': [100169, 100183, 100189, 100193, 100207],
 'Damian Joglar': [200003, 200009, 200013, 220003, 200023],
 'Aureliano Chavarría': [127031, 262151, 398039, 524287, 786433],
 'Luis Darío De Angeli': [],
 'Leandro.Cerutti': [],
 'Gustavo Denicolay': [],
 'Esteban Elia': [677213, 727817, 311237, 660719, 106427],
 'Karina L. Roitman': [201107, 433781, 590041, 604711, 931517],
 'Johanna Noval': [100333, 100999, 120011, 800029, 900091],
 'Milagros Rodríguez Saá': [100019, 200017, 300043, 400093, 500083],
 'Carla Fio

### Lista de numeros prohibidos

In [11]:
prohibidos = []

for key in v_dict:
    [prohibidos.append(item) for item in v_dict[key] if item not in prohibidos]
    

### 5 números primos

In [12]:
#Ahora genero basado en los existenes ocupados por otros
prohibidos
seeds = []
start = 100003
end = 999983

def es_primo(n):
  if n < 2:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True

def generar_primos(inicio, fin, prohibidos, cantidad=5):
  seeds = []
  while len(seeds) < cantidad:
    numero = random.randint(inicio, fin)
    if es_primo(numero) and numero not in seeds and numero not in prohibidos:
      seeds.append(numero)
  return seeds

generar_primos(start,end,prohibidos)

[794221, 635777, 976639, 125641, 677827]

### Verificación repetidos

In [13]:

repetidos = {}
v_list = []

for lista in v_dict.values():
    v_list.extend(lista)

for elemento in v_list:
        if v_list.count(elemento) > 1:
            if elemento in repetidos:
                repetidos[elemento] += 1
            else:
                repetidos[elemento] = 1

repetidos

{100019: 3, 100043: 2, 100049: 2, 100057: 2, 100069: 2, 100109: 2, 400093: 2}